In [140]:
# -*- coding: utf-8 -*-
import json
import cv2 as cv
# import cv2_imshow

# part 1:

def load_obj_each_frame(data_file):
  with open(data_file, 'r') as file:
    frame_dict = json.load(file)
  return frame_dict
def draw_target_object_center(video_file,obj_centers):
  count = 0
  cap = cv.VideoCapture(video_file)
  frames = []
  ok, image = cap.read()
  vidwrite = cv.VideoWriter("part_1_demo.mp4", cv.VideoWriter_fourcc(*'MP4V'), 30, (700,500))
  while ok:
    pos_x,pos_y = obj_centers[count]
    count+=1
    ######!!!!#######
    image = cv.resize(image, (700, 500)) # make sure your video is resize to this size, otherwise the coords in the data file won't work !!!
    ######!!!!#######
    image = cv.circle(image, (int(pos_x),int(pos_y)), 1, (0,0,255), 2)
    vidwrite.write(image)
    ok, image = cap.read()
  vidwrite.release()


frame_dict = load_obj_each_frame("object_to_track.json")
video_file = "commonwealth.mp4"
draw_target_object_center(video_file,frame_dict['obj'])

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [141]:
import numpy as np
import json
import cv2 as cv

In [142]:
file_path = 'part_1_object_tracking.json'

with open(file_path, 'r') as file:
    object_pos = json.load(file)

In [143]:
print(object_pos)

{'obj': [[-1, -1], [312, 228], [311, 228], [318, 205], [-1, -1], [-1, -1], [307, 225], [306, 224], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [287, 207], [287, 206], [286, 205], [286, 203], [-1, -1], [285, 202], [284, 202], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [278, 198], [276, 197], [-1, -1], [-1, -1], [275, 196], [-1, -1], [272, 194], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [262, 186], [261, 186], [260, 184], [260, 185], [259, 184], [258, 184], [257, 183], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [252, 176], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, 

In [144]:
def predict(x, P, A, Q):
    # Make sure that x is a 1D array with shape (2,)
    x = np.array(x).flatten()

    # Predict the next state
    x_pred = A.dot(x)

    # Predict the next covariance matrix
    P_pred = A.dot(P).dot(A.T) + Q

    return x_pred, P_pred


In [145]:
def update(x_pred, P_pred, z, H, R):
    # Ensure x_pred is a 2-element 1D array
    x_pred = np.array(x_pred).flatten()

    # Innovation or residual, should be a 2-element 1D array
    y = z - H.dot(x_pred)
    # System uncertainty, should be a 2x2 matrix
    S = H.dot(P_pred).dot(H.T) + R
    # Kalman gain, should be a 2x2 matrix
    K = P_pred.dot(H.T).dot(np.linalg.inv(S))
    # Updated state estimate, should be a 2-element 1D array
    x_update = x_pred + K.dot(y)
    I = np.eye(P_pred.shape[0])  # Identity matrix
    # Updated estimate uncertainty, should be a 2x2 matrix
    P_update = (I - K.dot(H)).dot(P_pred)

    return x_update, P_update


In [179]:
def generate_positions(obj_centers, num_passes):
    P = np.eye(2)
    A = np.eye(2) * 0.9972
    x = [312, 228]
    H = np.eye(2)
    Q = np.eye(2) * 0.01
    R = np.eye(2) * 1
    
    cap = cv.VideoCapture(video_file)
    
    positions = {}
    positions["obj"] = []

    frames = []
    ok, image = cap.read()
    
    forward = True
    for i in range(num_passes):
        count = 0
        if (not forward):
            x = frames[-1]
            frames = frames[::-1]
            A = np.eye(2) * 1.0028
            forward = True
        else:
            if (i != 0):
                x = frames[0]
            forward= False
            A = np.eye(2) * 0.9972
            
        while count < len(obj_centers):
            if(count == 0 and i== 0):
                count += 1
                continue
            else:
                pos_x, pos_y = obj_centers[count]
                count += 1
                if pos_x != -1 and pos_y != -1:
                    z = np.array([pos_x, pos_y])
                    if count == 1 and i == 0:
                        x = np.array([pos_x, pos_y])
                        P = np.diag([1, 1])
                    else:
                        x_pred, P_pred = predict(x, P, A, Q)  # predict next frame
                        x_update, P_update = update(x_pred, P_pred, z, H, R)
                        x_update = x_update.tolist()
                        frames.append([pos_x, pos_y])
                        positions["obj"] += [[pos_x, pos_y]]
                        x, P = x_update, P_update
                else:
                    x, P = predict(x, P, A, Q)
                    x = x.tolist()  # check the rounding issue
                    frames.append([x[0], x[1]])
                    positions["obj"] += [[x[0], x[1]]]
    
               
        if (not forward):
            frames = frames[::-1]
            
                
                
        


In [172]:
def draw_target_object_center(video_file, obj_centers, forward_pass = True):
    P = np.eye(2) 
    if(forward_pass):
        A = np.eye(2) * 0.9972
        x = [312, 228]
    else:
        A = np.eye(2) * 1.0028
        x = [155.48221359647187, 95.25131111414534]
    H = np.eye(2)
    Q = np.eye(2) * 0.01
    R = np.eye(2) * 1
    count = 0
    cap = cv.VideoCapture(video_file)
    
    
    positions = {}
    positions["obj"] = []
    
    frames = []  
    ok, image = cap.read()
    vidwrite = cv.VideoWriter("part_1_demo_with_kalman.mp4", cv.VideoWriter_fourcc(*'MP4V'), 30, (700,500))
    
    
    while ok:
        if(count == 0 and forward_pass):
            count += 1
            continue
        elif count == 249 and forward_pass:
            break
        # print(count)
        pos_x, pos_y = obj_centers[count]
        count += 1
        if pos_x != -1 and pos_y != -1:
            z = np.array([pos_x, pos_y])
            if count == 1:
                x = np.array([pos_x, pos_y])
                P = np.diag([1, 1])
            else:

                x_pred, P_pred = predict(x, P, A, Q) #predict next frame 

                x_update, P_update = update(x_pred, P_pred, z, H, R) #
                
                x_update = x_update.tolist()
                
                frames.append([pos_x, pos_y])
                positions["obj"] += [[pos_x, pos_y]]
                # print(x_update)
                image = cv.resize(image, (700, 500))
                # print(x_update[0], x_update[1])
                # if len(frames) > 1:
                #     cv.polylines(image, [np.array(frames)], False, (0, 0, 255), thickness=1)

                x, P = x_update, P_update
        else:
            # print(x)
            x, P = predict(x, P, A, Q)
            x = x.tolist() #check the rounding issue
            frames.append([x[0], x[1]])
            positions["obj"] += [[x[0], x[1]]]

            # cv.polylines(image, [np.array(frames)], False, (0, 0, 255), thickness=1)
        image = cv.resize(image, (700, 500))
        # image = cv.circle(image, (pos_x, pos_y), 1, (0, 0, 255), 2)

        vidwrite.write(image)
        ok, image = cap.read()
    vidwrite.release()
    if(forward_pass):
       filename = 'filled_positions.json'
    else:
        filename = 'backwards_filled_positions.json'
        frames = frames[::-1]
        positions["obj"] = positions["obj"][::-1]
    
    with open(filename, 'w') as f:
        json.dump(positions, f)




In [167]:
file_path = 'backwards_filled_positions.json'

with open(file_path, 'r') as file:
    object_pos = json.load(file)
    
print(len(object_pos['obj']))


249


In [178]:
file_path = 'part_1_object_tracking.json'

with open(file_path, 'r') as file:
    object_pos = json.load(file)

# draw_target_object_center(video_file, object_pos['obj'])
generate_positions(object_pos, 2)


IndexError: list index out of range

In [153]:

# back_pos = [[-1, -1]] + object_pos['obj']
# draw_target_object_center(video_file, back_pos[::-1], False)


Iteration 0
[312, 228]
[311, 228]
[318, 205]
[-1, -1]
[-1, -1]
[307, 225]
[306, 224]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[287, 207]
[287, 206]
[286, 205]
[286, 203]
[-1, -1]
[285, 202]
[284, 202]
[-1, -1]
[-1, -1]


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[278, 198]
[276, 197]
[-1, -1]
[-1, -1]
[275, 196]
[-1, -1]
[272, 194]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[262, 186]
[261, 186]
[260, 184]
[260, 185]
[259, 184]
[258, 184]
[257, 183]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[252, 176]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[232, 156]
[232, 156]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[225, 148]
[-1, -1]
[-1, -1]
[224, 148]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[219, 141]
[218, 141]
[217, 141]
[216, 139]
[216, 139]
[216, 139]
[216, 139]
[216, 138]
[-1, -1]
[213, 138]
[212,

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[166, 101]
[166, 100]
[167, 101]
[167, 101]
[167, 102]
[167, 102]
[168, 103]
[168, 103]
[169, 102]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[170, 107]
[-1, -1]
[-1, -1]
[171, 107]
[-1, -1]
[171, 108]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[177, 108]
[178, 108]
[177, 109]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[191, 120]
[190, 120]
[-1, -1]
[192, 120]
[193, 120]
[193, 120]
[193, 122]
[193, 121]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[-1, -1]
[198, 127]
[198, 127]
[199, 127]
[199, 128]
[-1, -1]
[202, 128]
[203, 128]
[203, 129]
[203, 130]
[203, 130]
[204, 130]
[205, 130]
[206, 131]
[205, 131]
[207, 131]
[207, 132]
[209, 132]
[-1, -1]
[211, 132]
[210, 132]
[211, 135]
[212, 135]
[211, 136]
[213, 136]
[212, 137]
[213, 138]
[-1, -1]

TypeError: cannot unpack non-iterable int object

In [12]:
video_file = "commonwealth.mp4"
draw_target_object_center(video_file, object_pos['obj'])

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


[312, 228]
[312, 228]
[312, 228]
[314, 222]
[314, 222]
[314, 222]
[313, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[308, 220]
[305, 218]
[302, 216]
[300, 214]
[300, 214]
[298, 213]
[296, 212]
[296, 212]
[296, 212]
[296, 212]
[296, 212]
[296, 212]
[296, 212]
[294, 210]
[292, 209]
[292, 209]
[292, 209]
[290, 208]
[290, 208]
[288, 207]
[288, 207]
[288, 207]
[288, 207]
[288, 207]
[288, 207]
[288, 207]
[288, 207]
[288, 207]
[288, 207]
[288, 207]
[288, 207]
[288, 207]
[285, 205]
[283, 203]
[281, 201]
[279, 199]
[277, 198]
[275, 197]
[273, 196]
[273, 196]
[273, 196]
[273, 196]
[273, 196]
[273, 196]
[273, 196]
[273, 196]
[273, 196]
[273, 196]
[273, 196]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]

In [161]:
def create_video_path(video_file, obj_centers):
    obj_centers = [[int(round(x[0])), int(round(x[1]))] for x in obj_centers]
    count = 0
    cap = cv.VideoCapture(video_file)
    ok, image = cap.read()
    vidwrite = cv.VideoWriter(
        "part_1_demo_with_kalman.mp4", cv.VideoWriter_fourcc(*'MP4V'), 30, (700, 500))

    while ok:
        if(count == 0):
            count += 1
            continue
        elif count == 248:
            break
        # print(count)
        pos_x, pos_y = obj_centers[count]
        count += 1
        image = cv.resize(image, (700, 500))
        # print([np.array(obj_centers[:count+1])])
        cv.polylines(image, [np.array(obj_centers[:count+1])],
                     False, (0, 0, 255), thickness=2)
        image = cv.circle(image, (pos_x, pos_y), 1, (0, 0, 255), 2)        
        vidwrite.write(image)
        ok, image = cap.read()
    vidwrite.release()



In [162]:
file_path = 'backwards_filled_positions.json'

with open(file_path, 'r') as file:
    object_pos = json.load(file)
    

create_video_path(video_file, object_pos['obj'])


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [7]:
print(len(object_pos["obj"]))


248


In [84]:

# part 2:

def draw_object(object_dict,image,color = (0, 255, 0), thickness = 2,c_color= \
                (255, 0, 0)):
  # draw box
  x = object_dict['x_min']
  y = object_dict['y_min']
  width = object_dict['width']
  height = object_dict['height']
  image = cv.rectangle(image, (x, y), (x + width, y + height), color, thickness)
  return image

def draw_objects_in_video(video_file,frame_dict):
  count = 0
  cap = cv.VideoCapture(video_file)
  frames = []
  ok, image = cap.read()
  vidwrite = cv.VideoWriter("part_2_demo.mp4", cv.VideoWriter_fourcc(*'MP4V'), 30, (700,500))
  while ok:
    ######!!!!#######
    image = cv.resize(image, (700, 500)) # make sure your video is resize to this size, otherwise the coords in the data file won't work !!!
    ######!!!!#######
    obj_list = frame_dict[str(count)]
    for obj in obj_list:
      image = draw_object(obj,image)
    vidwrite.write(image)
    count+=1
    ok, image = cap.read()
  vidwrite.release()

frame_dict = load_obj_each_frame("frame_dict.json")
video_file = "commonwealth.mp4"
draw_objects_in_video(video_file,frame_dict)

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [135]:
class MultiObjKalmanFilter:
    def __init__(self, ):
        self.towards_A_matrix = np.eye(2) * 1.0028
        self.away_A_matrix = np.eye(2) * 0.9972
        self.stationary_A_matrix = np.eye(2)
        self.H = np.eye(2)
        self.Q = np.eye(2) * 0.01
        self.R = np.eye(2) * 1
        self.A = self.stationary_A_matrix  # Default A matrix
        self.x = np.zeros(2)  # Initial state estimate
        self.P = np.eye(2)  # Initial covariance estimate
        self.x_last = self.x

    def set_motion_status(self):
        result = np.around(self.x) - self.x_last
        if np.all(result > 0):
            self.A = self.towards_A_matrix
        elif np.all(result < 0):
            self.A = self.away_A_matrix
        elif np.all(result = 0):
            self.A = self.stationary_A_matrix
        else:
            raise ValueError("Invalid motion status")

    def update(self, z):
        self.x_last = z
        y = z - self.H.dot(self.x)
        S = self.H.dot(self.P).dot(self.H.T) + self.R
        K = self.P.dot(self.H.T).dot(np.linalg.inv(S))
        self.x = self.x + K.dot(y)
        I = np.eye(self.P.shape[0])
        self.P = (I - K.dot(self.H)).dot(self.P)
        return self.x, self.P

    def predict(self):
        self.x = self.A.dot(self.x)
        self.P = self.A.dot(self.P).dot(self.A.T) + self.Q
        return self.x


In [58]:
#Update average width/height of a detected object

def update_known_object_characteristics(id, object):
    if id not in detected_obj_avg_size:
        # If not, initialize the record for this object ID
        detected_obj_avg_size[id] = {
            'num_updates': 1,
            'total_height': object['height'],
            'total_width': object['width']
        }
    else:
        detected_obj_avg_size[id]['num_updates'] += 1
        detected_obj_avg_size[id]['total_height'] += object['height']
        detected_obj_avg_size[id]['total_width'] += object['width']

    detected_obj_avg_size[id]['avg_height'] = detected_obj_avg_size[id]['total_height'] / detected_obj_avg_size[id]['num_updates']
    detected_obj_avg_size[id]['avg_width'] = detected_obj_avg_size[id]['total_width'] / detected_obj_avg_size[id]['num_updates']

def get_similar_sized_object(obj):
    min_diff = float('inf') 
    similar_obj_id = None
    
    for detected_obj in detected_obj_avg_size:
        height_diff = abs(detected_obj['avg_height'] - obj['height'])
        width_diff = abs(detected_obj['avg_width'] - obj['width'])
        combined_diff = height_diff + width_diff  
        
        if combined_diff < min_diff:
            min_diff = combined_diff
            similar_obj_id = detected_obj['id']
    
    return similar_obj_id


def are_coordinates_close(coord1, coord2, margin):
   
    distance = np.linalg.norm(np.array(coord1) - np.array(coord2))

    return distance <= margin





In [139]:
# initialization step - give every object a unique ID 
file_path = 'part_2_frame_dict.json'

with open(file_path, 'r') as file:
    obj_positions = json.load(file)


#set the default ID to -1
for frame in obj_positions.values():
    for obj in frame:
        obj['id'] = -1

#keeps track of the size of previously ID'd objects, used to compare with the size of unkown objects to help
#with identification
# detected_obj_avg_size = {}


#known objects
prev_positions = obj_positions["0"]

#measurements
curr_positions = obj_positions["1"]

#ID Numtracker:
newest_id = 0

pixel_threshold = 15


for i in range(len(prev_positions)):
    prev_positions[i]['id'] = i+1
    newest_id += 1;

detected_obj_tracks = {}
detected_obj_kalmans = {}


#initialize kalman filters for all existing objects
for object in prev_positions:
    detected_obj_kalmans[object['id']] = MultiObjKalmanFilter()
    detected_obj_kalmans[object['id']].x = np.array((object["x_min"], object["y_min"]))
    detected_obj_tracks[object['id']] = object



In [138]:


for frame in list(obj_positions.keys())[1:]:
    curr_positions = obj_positions[frame]
    
    #loop handling old objects: assigning to new detections, or ruling them out as having left the frame
    for object in prev_positions:
        matched = False
        obj_estimates = [] #array containing the 3 possible estimated locations of the obj
        
        #make an estimation assuming motion towards the camera
        detected_obj_kalmans[object['id']].set_motion_status()
        towards_estimation = detected_obj_kalmans[object['id']].predict()
        obj_estimates.append(towards_estimation)
        
        # #make an estimation assuming motion away from the camera
        # detected_obj_kalmans[object['id']].set_motion_status("away")
        # away_estimation = detected_obj_kalmans[object['id']].predict()
        # obj_estimates.append(away_estimation)
        
        # #make an estimation assuming motion stationary to the camera
        # detected_obj_kalmans[object['id']].set_motion_status("stationary")
        # stationairy_estimation = detected_obj_kalmans[object['id']].predict()
        # obj_estimates.append(stationairy_estimation)
        
        for new_det in curr_positions: #for every newly detected obj
            if matched:
                break
            for i in range(len(obj_estimates)): #for all 3 estimations
                # check if model finds a match
                if are_coordinates_close(obj_estimates[i].tolist(), [new_det["x_min"], new_det["y_min"]], pixel_threshold):
            
                    
                    new_det['id'] = object['id'] #link the new detection to an existing object
                    
                    #update the kalman filter instance's estimation with the new measurment, now confirmed to be for that object
                    detected_obj_kalmans[object['id']].update(
                        np.array([new_det["x_min"], new_det["y_min"]]))
                    
                    # if match, that object now has a set model for the future
                    detected_obj_kalmans[object['id']
                                         ].set_motion_status()
                    matched = True
                else:
                    continue 
            
            
            
    #loop handling new objects that have not found a match from previous frame (could either be new objects or objects that already exist but were
    # not in the previous frame for whatever reason)
    for new_detection in curr_positions:
        if new_detection['id'] == -1: #if obj not classified TODO: change the default ID to -1 to avoid errors
            matched = False
            for old_obj in detected_obj_tracks.values():
                if are_coordinates_close(np.array([new_detection["x_min"], new_detection["y_min"]]), np.array([old_obj["x_min"], old_obj["y_min"]]), pixel_threshold):
                    #the "new" object was actually an old object that we lost track of in the last frame
                    new_detection['id'] = old_obj['id'] #update it's ID and kalman filters accordingly 
                    detected_obj_kalmans[object['id']].update(
                        np.array([new_detection["x_min"], new_detection["y_min"]]))
                    matched = True
            
            if (not matched): #the new object was not detected in the last frame or any previous frames, create a new object ID for it 
                newest_id += 1  # create a new ID for the new obj and assign
                new_detection['id'] = newest_id
                #create its kalman filter instance and initialize it
                detected_obj_kalmans[new_detection['id']] = MultiObjKalmanFilter()
                detected_obj_kalmans[new_detection['id']].x = np.array(
                    [new_detection["x_min"], new_detection["y_min"]])


    for new_detection in curr_positions:
    #update the object database withe the new coordinates of all currently tracked objects
        detected_obj_tracks[new_detection['id']
                            ] = new_detection
    
    
    #update all filters anyways (incase we lost track of an object that will re-appear later)
    for object in detected_obj_kalmans.values():
        object.predict()
        
        
    #the curr_positions dictionairy now contains identified objects
    obj_positions[frame] = curr_positions
    prev_positions = curr_positions
    

file_path = "obj_detections_test.json"

# Save the dictionary as a JSON file
with open(file_path, "w") as json_file:
    json.dump(obj_positions, json_file)

file_path


TypeError: all() got an unexpected keyword argument 'result'

In [54]:
# for each new frame

    #for each old object
        #estimate 3 possible locations
            #assuming:
                #stationairy model
                #towards model
                #away model
            
            #get 3 coord positions
            
        #for each new detection:
            #check for each prediction to see if match 
                #if match: 
                    #go to frame dict [frame#]: find box with matching coord, set ID 
                    # update estimations for that object 
                #else no match:
                    #assume object has left frame 

    #for each new detection:
        #if detection not given an ID
        #give new ID 
        
    #set old_detections = new_detections
    
    

#loop to next frame
        


Given input array:
[[15 15  0]
 [ 0  0 10]
 [ 5  5  0]]

m2 array (max num of zeroes from horizontal vs vertical) (- for horizontal and + for vertical):
[[ 1  1  2]
 [-2 -2  2]
 [ 1  1  2]]

Lines array:
[[0 0 0]
 [0 0 0]
 [0 0 0]]


In [127]:
import cv2 as cv
import json


def load_obj_each_frame(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    return data

def draw_object(object_dict, image, color=(0, 255, 0), thickness=2, c_color=(255, 0, 0)):
  # draw box
  x = object_dict['x_min']
  y = object_dict['y_min']
  width = object_dict['width']
  height = object_dict['height']
  id = object_dict['id']
  image = cv.rectangle(
      image, (x, y), (x + width, y + height), color, thickness)
  cv.putText(image, str(id), (x, y - 10),
             cv.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
  return image

def draw_objects_in_video(video_file, frame_dict):
  count = 0
  cap = cv.VideoCapture(video_file)
  frames = []
  ok, image = cap.read()
  vidwrite = cv.VideoWriter(
      "part_2_demo.mp4", cv.VideoWriter_fourcc(*'MP4V'), 30, (700, 500))
  while ok:
    ######!!!!#######
    # make sure your video is resize to this size, otherwise the coords in the data file won't work !!!
    image = cv.resize(image, (700, 500))
    ######!!!!#######
    obj_list = frame_dict[str(count)]
    for obj in obj_list:
      image = draw_object(obj, image)
    vidwrite.write(image)
    count += 1
    ok, image = cap.read()
  vidwrite.release()


# Assuming the JSON file is named "frame_dict.json" and the video file is named "commonwealth.mp4"
frame_dict = load_obj_each_frame("obj_detections_test.json")
video_file = "commonwealth.mp4"
draw_objects_in_video(video_file, frame_dict)


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
